## Setup

In [4]:
#imports
import numpy as np
import xarray as xr
import torch
from src.train_nn_pytorch import Dataset, init_torch_device
from torch.utils.data import RandomSampler

#directories
datadir = '/gpfs/work/nonnenma/data/forecast_predictability/weatherbench/5_625deg/'
res_dir = '/gpfs/work/nonnenma/results/forecast_predictability/weatherbench/5_625deg/'

## Define options

In [18]:
lead_time = 3 * 24  # hours in the future for predictions
batch_size = 32

train_years = ('1979', '2015')
valid_years = ('2016', '2016')
test_years = ('2017', '2018')

var_dict =  {'geopotential': ('z', [100, 200, 500, 850, 1000]), 
             'temperature': ('t', [100, 200, 500, 850, 1000]), 
             'u_component_of_wind': ('u', [100, 200, 500, 850, 1000]), 
             'v_component_of_wind': ('v', [100, 200, 500, 850, 1000]), 
             'constants': ['lsm','orography','lat2d']}

In [6]:
device = init_torch_device()

CUDA not available


## Load data

In [ ]:
ds = xr.merge(
            [xr.open_mfdataset(f'{datadir}/{var}/*.nc', combine='by_coords')
             for var in var_dict.keys()],
            fill_value=0  # For the 'tisr' NaNs
        )

In [19]:
ds_train = ds.sel(time=slice(*train_years))
ds_valid = ds.sel(time=slice(*valid_years))
ds_test = ds.sel(time=slice(*test_years))